# Q1

In [2]:
HC = 990
Cancer = 10

In [4]:
TN = 990
FN = 10
FP = 0
TP = 0
n = 1000

In [5]:
Accuracy = (TP + TN)/n
sensitivity = TP/(TP+FN)
sepcificity = TN/(TN+FP)
precision = TP/(TP+FP)

ZeroDivisionError: division by zero

# Q2

In [37]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [7]:
PPMI = pd.read_csv('E:\ML in neuro\FC_Mohammad.csv')

In [8]:
PPMI

,Subject,Group,Sex,Age,lLPFC and rPPC,lLPFC and lIFG,lLPFC and rIFG,lLPFC and lpSTG,lLPFC and rpSTG,lPPC and rLPFC,...,rPPC and lIFG,rPPC and rIFG,rPPC and lpSTG,rPPC and rpSTG,lIFG and rIFG,lIFG and lpSTG,lIFG and rpSTG,rIFG and lpSTG,rIFG and rpSTG,lpSTG and rpSTG
0,1,PD,M,71,-0.103246,0.350126,0.394340,-0.576904,-0.039407,-0.465657,...,0.586340,0.123240,1.116462,1.991402,-0.202862,0.191020,0.700737,-0.243791,0.011227,1.094555
1,2,PD,M,75,0.321440,0.384504,-1.041718,0.199509,0.155007,0.151488,...,0.660146,-0.506504,0.545130,1.194057,-0.113396,0.578819,0.277461,-0.127428,-0.469112,0.802674
2,3,PD,F,54,-0.060092,0.348650,-0.115624,0.025965,-0.537761,0.875136,...,1.034337,0.650386,1.076177,0.532748,0.908516,1.403653,0.624336,1.218256,0.901400,0.803128
3,4,PD,M,67,-2.048700,1.145381,-0.138926,-0.310826,-1.264406,0.613881,...,-0.786227,0.328857,0.142990,1.312452,0.356541,-0.701987,-0.695071,-1.386374,-0.051156,0.563862
4,5,PD,F,61,0.104956,0.450186,0.126491,0.160993,-0.853711,0.766645,...,-0.052218,0.376734,0.294777,-0.265599,0.676890,-0.416838,-0.749584,-0.910490,-0.431458,-0.022669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,142,Control,F,65,0.208877,0.527658,-0.121582,0.890016,-0.689229,0.169960,...,-0.405886,0.069582,0.678912,0.181889,0.294484,-0.123988,-0.245210,-0.294329,-0.210474,-0.279567
142,143,Control,F,66,0.145393,0.416814,-0.705539,-0.971823,0.096652,0.343644,...,0.797781,0.648840,0.000723,0.394908,0.446373,0.090323,0.547290,0.966238,0.470560,0.486163
143,144,Control,M,72,0.070322,-0.165584,0.253232,0.400744,0.409499,0.316525,...,-0.000253,0.310873,0.522322,0.322046,0.648687,0.716026,0.936013,1.556792,1.211635,1.894386
144,145,Control,F,30,0.969361,-0.583140,-0.764163,-0.735485,-0.873675,-0.118595,...,-0.813304,-1.121430,-0.927283,-0.942936,1.004450,0.803312,0.786546,1.733911,1.665241,1.206069


In [9]:
PPMI['Sex'] = PPMI['Sex'].str.strip()
PPMI['Sex'] = PPMI['Sex'].replace({'M': 1, 'F': 2})
PPMI['Sex'] = PPMI['Sex'].astype(int)
PPMI['Group'] = PPMI['Group'].str.strip()
PPMI['Group'] = PPMI['Group'].replace({'PD': 1, 'Control': 2})
PPMI['Group'] = PPMI['Group'].astype(int)

In [10]:
y = PPMI['Group']

In [11]:
features =PPMI.loc[:, 'lLPFC and rPPC':'lpSTG and rpSTG']

# outlier and scaling

In [12]:
for feature in PPMI.columns:
    Q1= PPMI[feature].quantile(0.25)
    Q3= PPMI[feature].quantile(0.75)
    IQR = Q3 - Q1
    low_bound = Q1 - 1.5 * IQR
    up_bound = Q3 + 1.5 * IQR
    PPMI[feature] = np.where(
        (PPMI[feature] < low_bound )| (PPMI[feature] > up_bound), 
        PPMI[feature].median(),
        PPMI[feature])

In [13]:
stn = StandardScaler()
PPMI.iloc[: , 4: ] =  stn.fit_transform(PPMI.iloc[: , 4:])

In [16]:
y = PPMI['Group']
X =PPMI.loc[:, 'lLPFC and rPPC':'lpSTG and rpSTG']

# test and train

In [17]:
X_train , X_test, y_train , y_test =  train_test_split(X,y, test_size = 0.2, random_state = 6)

# feeature selection

In [18]:
correlation_matrix = X_train.corrwith(y_train)

In [19]:
correlation_matrix

lLPFC and rPPC    -0.023376
lLPFC and lIFG    -0.188330
lLPFC and rIFG    -0.072457
lLPFC and lpSTG   -0.101960
lLPFC and rpSTG   -0.018713
lPPC and rLPFC    -0.068516
lPPC and rPPC     -0.005718
lPPC and lIFG     -0.063392
lPPC and rIFG     -0.109058
lPPC and lpSTG     0.026336
lPPC and rpSTG     0.117577
rLPFC and rPPC     0.163231
rLPFC and lIFG     0.088298
rLPFC and rIFG     0.153567
rLPFC and lpSTG   -0.193848
rLPFC and rpSTG   -0.059399
rPPC and lIFG     -0.103920
rPPC and rIFG     -0.110027
rPPC and lpSTG     0.026092
rPPC and rpSTG     0.005171
lIFG and rIFG      0.029248
lIFG and lpSTG    -0.020153
lIFG and rpSTG     0.008075
rIFG and lpSTG     0.035356
rIFG and  rpSTG    0.046593
lpSTG and rpSTG    0.068805
dtype: float64

In [27]:
X_train

,lLPFC and rPPC,lLPFC and lIFG,lLPFC and rIFG,lLPFC and lpSTG,lLPFC and rpSTG,lPPC and rLPFC,lPPC and rPPC,lPPC and lIFG,lPPC and rIFG,lPPC and lpSTG,...,rPPC and lIFG,rPPC and rIFG,rPPC and lpSTG,rPPC and rpSTG,lIFG and rIFG,lIFG and lpSTG,lIFG and rpSTG,rIFG and lpSTG,rIFG and rpSTG,lpSTG and rpSTG
60,-0.963225,1.068454,1.569929,1.737421,0.389299,-0.621842,-1.093263,1.689717,1.200140,0.643431,...,-0.734753,-0.494499,-0.657154,-0.653422,-0.077120,0.663702,-0.933669,0.937301,0.552727,-1.296101
53,-0.957108,1.686782,-0.829085,-1.451647,-0.706513,0.213680,0.137272,0.644595,-1.213902,0.182054,...,-0.439091,0.150111,0.088504,0.715029,-0.050482,-1.375505,-0.821077,-0.580476,-0.395900,0.420979
84,-0.476751,0.711701,0.906415,-0.077650,-0.416273,-1.286273,2.477001,-1.292577,0.828359,-0.286773,...,-0.887488,1.055366,0.151144,0.575003,-0.896062,-1.174093,-1.541264,0.843383,-0.000410,0.157098
52,0.288418,0.385695,-0.523256,-0.613453,-1.119410,1.996105,-0.009065,0.305678,-0.979621,-1.152320,...,-1.071135,-2.154724,-1.056998,-1.106864,0.113825,-0.422824,-0.664002,-0.389590,-0.412410,1.071644
17,2.181053,-0.239732,0.011083,1.001635,1.934201,1.502940,-0.755822,-1.259289,-1.181925,1.612376,...,0.753911,0.654576,0.403366,2.046869,1.498094,-1.053230,0.439859,-1.155633,-0.244876,-0.922669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,1.428754,-1.388253,-1.229817,-1.476944,-1.362522,-0.465340,0.352388,-0.736978,-1.553890,-2.033835,...,-1.528120,-2.347177,-2.068402,-2.216142,0.847631,0.458180,0.974061,2.113890,2.073520,0.637640
79,0.698345,-1.144796,0.505838,1.686935,1.645518,0.003535,0.686544,-0.378674,-0.168300,0.707592,...,-0.099104,-0.518906,1.410301,-0.132309,-0.876318,-1.232370,-1.454928,-0.385528,0.287777,0.243090
109,0.963328,0.083104,-0.819741,-1.173872,-1.047884,0.422142,-0.604461,0.059556,-0.558105,-0.502156,...,1.093659,0.556950,-0.563972,-0.531953,0.119050,-0.685433,-0.652712,0.288594,0.647467,0.143054
106,0.075078,0.321462,0.382884,0.172768,0.531259,0.046335,-0.368546,-0.233147,-0.441954,-0.387283,...,-1.013659,-0.688445,-0.139127,0.425653,1.373975,-0.713011,-1.087251,-0.372045,-1.015305,-0.349126


In [33]:
X_train = X_train[['lPPC and rpSTG','rLPFC and rPPC','rLPFC and rIFG']]

In [34]:
X_train

,lPPC and rpSTG,rLPFC and rPPC,rLPFC and rIFG
60,-0.347249,-0.029291,0.456130
53,0.917876,-0.825196,-1.256090
84,0.605589,-1.414338,0.290091
52,-1.386532,0.687284,-1.151828
17,-0.067517,0.206814,-0.808184
...,...,...,...
144,-1.880615,-0.314013,-0.666846
79,0.213587,-1.035718,1.033337
109,-0.491646,2.279895,0.613291
106,0.496557,0.626880,0.796827


# SVM

In [36]:
svm = SVC(C = 0.1, kernel = 'poly', degree = 3)
svm.fit(X_train, y_train)
y_predict_train = svm.predict(X_train)

In [38]:
print(classification_report(y_train, y_predict_train))

              precision    recall  f1-score   support

         1.0       0.88      0.13      0.23        52
         2.0       0.58      0.98      0.73        64

    accuracy                           0.60       116
   macro avg       0.73      0.56      0.48       116
weighted avg       0.71      0.60      0.51       116



In [44]:
svm = SVC(C = 0.1, kernel = 'poly', degree = 4)
svm.fit(X_train, y_train)
y_predict_train = svm.predict(X_train)
print(classification_report(y_train, y_predict_train))

              precision    recall  f1-score   support

         1.0       0.80      0.15      0.26        52
         2.0       0.58      0.97      0.73        64

    accuracy                           0.60       116
   macro avg       0.69      0.56      0.49       116
weighted avg       0.68      0.60      0.52       116



In [47]:
svm = SVC(C = 1, kernel = 'poly', degree = 4)
svm.fit(X_train, y_train)
y_predict_train = svm.predict(X_train)
print(classification_report(y_train, y_predict_train))

              precision    recall  f1-score   support

         1.0       0.73      0.21      0.33        52
         2.0       0.59      0.94      0.73        64

    accuracy                           0.61       116
   macro avg       0.66      0.57      0.53       116
weighted avg       0.66      0.61      0.55       116



In [45]:
svm = SVC(C = 0.1, kernel = 'poly', degree = 5)
svm.fit(X_train, y_train)
y_predict_train = svm.predict(X_train)
print(classification_report(y_train, y_predict_train))

              precision    recall  f1-score   support

         1.0       1.00      0.21      0.35        52
         2.0       0.61      1.00      0.76        64

    accuracy                           0.65       116
   macro avg       0.80      0.61      0.55       116
weighted avg       0.78      0.65      0.57       116



In [48]:
svm = SVC(C = 10, kernel = 'poly', degree = 5)
svm.fit(X_train, y_train)
y_predict_train = svm.predict(X_train)
print(classification_report(y_train, y_predict_train))

              precision    recall  f1-score   support

         1.0       0.88      0.44      0.59        52
         2.0       0.68      0.95      0.79        64

    accuracy                           0.72       116
   macro avg       0.78      0.70      0.69       116
weighted avg       0.77      0.72      0.70       116



In [46]:
svm = SVC(C = 0.1, kernel = 'poly', degree = 6)
svm.fit(X_train, y_train)
y_predict_train = svm.predict(X_train)
print(classification_report(y_train, y_predict_train))

              precision    recall  f1-score   support

         1.0       0.92      0.23      0.37        52
         2.0       0.61      0.98      0.75        64

    accuracy                           0.65       116
   macro avg       0.77      0.61      0.56       116
weighted avg       0.75      0.65      0.58       116



In [40]:
svm = SVC(C = 0.1,  kernel= 'linear')
svm.fit(X_train, y_train)
y_predict_train = svm.predict(X_train)
print(classification_report(y_train, y_predict_train))


              precision    recall  f1-score   support

         1.0       0.64      0.31      0.42        52
         2.0       0.60      0.86      0.71        64

    accuracy                           0.61       116
   macro avg       0.62      0.58      0.56       116
weighted avg       0.62      0.61      0.58       116



In [41]:
svm = SVC(C = 1,  kernel= 'linear')
svm.fit(X_train, y_train)
y_predict_train = svm.predict(X_train)
print(classification_report(y_train, y_predict_train))


              precision    recall  f1-score   support

         1.0       0.58      0.42      0.49        52
         2.0       0.62      0.75      0.68        64

    accuracy                           0.60       116
   macro avg       0.60      0.59      0.58       116
weighted avg       0.60      0.60      0.59       116



In [42]:
svm = SVC(C = 10,  kernel= 'linear')
svm.fit(X_train, y_train)
y_predict_train = svm.predict(X_train)
print(classification_report(y_train, y_predict_train))


              precision    recall  f1-score   support

         1.0       0.58      0.50      0.54        52
         2.0       0.63      0.70      0.67        64

    accuracy                           0.61       116
   macro avg       0.61      0.60      0.60       116
weighted avg       0.61      0.61      0.61       116



In [43]:
svm = SVC(C = 100,  kernel= 'linear')
svm.fit(X_train, y_train)
y_predict_train = svm.predict(X_train)
print(classification_report(y_train, y_predict_train))


              precision    recall  f1-score   support

         1.0       0.58      0.50      0.54        52
         2.0       0.63      0.70      0.67        64

    accuracy                           0.61       116
   macro avg       0.61      0.60      0.60       116
weighted avg       0.61      0.61      0.61       116



In [49]:
svm = SVC(C = 0.1,  kernel= 'rbf', gamma = 'scale')
svm.fit(X_train, y_train)
y_predict_train = svm.predict(X_train)
print(classification_report(y_train, y_predict_train))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00        52
         2.0       0.55      1.00      0.71        64

    accuracy                           0.55       116
   macro avg       0.28      0.50      0.36       116
weighted avg       0.30      0.55      0.39       116



D:\anacondaa\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anacondaa\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anacondaa\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [51]:
svm = SVC(C = 0.1,  kernel= 'rbf', gamma = 'auto')
svm.fit(X_train, y_train)
y_predict_train = svm.predict(X_train)
print(classification_report(y_train, y_predict_train))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00        52
         2.0       0.55      1.00      0.71        64

    accuracy                           0.55       116
   macro avg       0.28      0.50      0.36       116
weighted avg       0.30      0.55      0.39       116



D:\anacondaa\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anacondaa\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anacondaa\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
svm = SVC(C = 0.1,  kernel= 'rbf', gamma = 0.1)
svm.fit(X_train, y_train)
y_predict_train = svm.predict(X_train)
print(classification_report(y_train, y_predict_train))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00        52
         2.0       0.55      1.00      0.71        64

    accuracy                           0.55       116
   macro avg       0.28      0.50      0.36       116
weighted avg       0.30      0.55      0.39       116



D:\anacondaa\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anacondaa\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anacondaa\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# poly degree 5 with c = 10 seems to work best.
# i would expect using a very low C and very high degree of poly would cuase overfitting yet it didn't happen.
#error while using rbf kernel is weird.
#in this example percision of .88 for 1 means that out of 100 we'll have 12 wrongly assumed group 1
#accuracy of 0.72 means that in total we have 72% chance of correctly choosing group for our subject no matter the group.
#recall; can the model detect all the instances of a group correctly?

In [57]:
svm = SVC(C = 10, kernel = 'poly', degree = 5)
svm.fit(X_train, y_train)
y_predict_train = svm.predict(X_train)
print(classification_report(y_train, y_predict_train))

              precision    recall  f1-score   support

         1.0       0.88      0.44      0.59        52
         2.0       0.68      0.95      0.79        64

    accuracy                           0.72       116
   macro avg       0.78      0.70      0.69       116
weighted avg       0.77      0.72      0.70       116



In [60]:
svm = SVC(C = 0.001, kernel = 'poly', degree = 20)
svm.fit(X_train, y_train)
y_predict_train = svm.predict(X_train)
print(classification_report(y_train, y_predict_train))

              precision    recall  f1-score   support

         1.0       1.00      0.27      0.42        52
         2.0       0.63      1.00      0.77        64

    accuracy                           0.67       116
   macro avg       0.81      0.63      0.60       116
weighted avg       0.79      0.67      0.62       116



# according to test data 

In [58]:
X_test = X_test[['lPPC and rpSTG','rLPFC and rPPC','rLPFC and rIFG']]

In [59]:
y_predict_test = svm.predict(X_test)
print(classification_report(y_test, y_predict_test))

              precision    recall  f1-score   support

         1.0       0.56      0.28      0.37        18
         2.0       0.38      0.67      0.48        12

    accuracy                           0.43        30
   macro avg       0.47      0.47      0.43        30
weighted avg       0.49      0.43      0.42        30

